<a href="https://colab.research.google.com/github/gunman-vagabond/python_colaboratory/blob/master/gunmanRecognitionChainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -r images
!rm images.zip*
!wget http://fc.jpn.org/ryuba/gunman/keras/gunmanRecognition/images.zip -O images.zip
!unzip images.zip


rm: cannot remove 'images': No such file or directory
rm: cannot remove 'images.zip*': No such file or directory
--2019-03-11 22:49:29--  http://fc.jpn.org/ryuba/gunman/keras/gunmanRecognition/images.zip
Resolving fc.jpn.org (fc.jpn.org)... 59.106.13.159
Connecting to fc.jpn.org (fc.jpn.org)|59.106.13.159|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14575997 (14M) [application/zip]
Saving to: ‘images.zip’

images.zip          100%[===================>]  13.90M  4.24MB/s    in 3.3s    

2019-03-11 22:49:32 (4.24 MB/s) - ‘images.zip’ saved [14575997/14575997]

Archive:  images.zip
   creating: images/gunman/
  inflating: images/gunman/gunman000_images_10_.jpg  
  inflating: images/gunman/gunman000_images_101_.jpg  
  inflating: images/gunman/gunman000_images_104_.jpg  
  inflating: images/gunman/gunman000_images_105_.jpg  
  inflating: images/gunman/gunman000_images_11_.jpg  
  inflating: images/gunman/gunman000_images_111_.jpg  
  inflating: images/gunman/gun

In [0]:
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import Variable,Chain,optimizers,serializers,datasets

import numpy as np
import matplotlib.pyplot as plt

from chainer.datasets import tuple_dataset
from chainer import training, iterators
from chainer.training import extensions

import keras
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from PIL import Image
import glob



Using TensorFlow backend.


In [0]:
folder = ["images/gunman", "images/ultraman", "images/rider", "images/precure"]
image_size = 50

X = []
Y = []
for index, name in enumerate(folder):
    dir = "./" + name
    files = glob.glob(dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

X = np.array(X)
Y = np.array(Y)

X = X.astype('float32')
X = X / 255.0

#print(Y)

#Y = np_utils.to_categorical(Y, 4)

#print(Y)

#X_train = X
#Y_train = Y

#print (X)
#plt.imshow(X_train[100])
#plt.show()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

X_train = X_train.reshape(len(X_train), 3, 50, 50)
X_test = X_test.reshape(len(X_test), 3, 50, 50)

#X_train.reshape(len(X_train), 50, 50, 3)




In [0]:
class Alex(Chain):
	# AlexNet
	def __init__(self):
	    super(Alex, self).__init__(
	        conv1 = L.Convolution2D(n_channel, 96, 11, stride=4),
	        conv2 = L.Convolution2D(96, 256, 5, pad=2),
	        conv3 = L.Convolution2D(256, 384, 3, pad=1),
	        conv4 = L.Convolution2D(384, 384, 3, pad=1),
	        conv5 = L.Convolution2D(384, 256, 3, pad=1),
	        fc6 = L.Linear(None, 4096),
	        fc7 = L.Linear(4096, 4096),
	        fc8 = L.Linear(4096, n_label),
	    )

	def __call__(self, x):
		h = F.max_pooling_2d(F.local_response_normalization(
		    F.relu(self.conv1(x))), 3, stride=2)
		h = F.max_pooling_2d(F.local_response_normalization(
		    F.relu(self.conv2(h))), 3, stride=2)
		h = F.relu(self.conv3(h))
		h = F.relu(self.conv4(h))
		h = F.max_pooling_2d(F.relu(self.conv5(h)), 2, stride=2)
		h = F.dropout(F.relu(self.fc6(h)))
		h = F.dropout(F.relu(self.fc7(h)))
		return self.fc8(h)

class DeepLearningClassifier:
	def __init__(self):
		model = Alex()
		self.model = L.Classifier(model)
		self.opt = optimizers.Adam()
		self.opt.setup(self.model)

	def fit(self,X_train, y_train):
		train_data = tuple_dataset.TupleDataset(X_train, y_train)
		train_iter = chainer.iterators.SerialIterator(train_data, batch_size)
		updater = chainer.training.StandardUpdater(train_iter, self.opt)
		self.trainer = chainer.training.Trainer(updater, (n_epoch, 'epoch'), out='result')
		self.trainer.extend(extensions.LogReport())
		self.trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy']))
		self.trainer.extend(extensions.ProgressBar())
		self.trainer.run()

	def fit_and_score(self, X_train, y_train, X_test, y_test):
		train_data = tuple_dataset.TupleDataset(X_train, y_train)
		test_data = tuple_dataset.TupleDataset(X_test, y_test)
		train_iter = chainer.iterators.SerialIterator(train_data, batch_size)
		test_iter = chainer.iterators.SerialIterator(test_data, batch_size, repeat=False, shuffle=False)
		updater=chainer.training.StandardUpdater(train_iter, self.opt)
		self.trainer = chainer.training.Trainer(updater, (n_epoch, 'epoch'), out='result')
		self.trainer.extend(extensions.Evaluator(test_iter, self.model))
		self.trainer.extend(extensions.LogReport())
		self.trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy']))
		self.trainer.extend(extensions.ProgressBar())
		self.trainer.run()

	def predict(self, X_test):
		x=Variable(X_test)
		y=self.model.predictor(x)
		answer=y.data
		answer=np.argmax(answer, axis=1)
		return answer

	def score(self, X_test, y_test):
		y=self.predict(X_test)
		N=y_test.size
		return 1.0-np.count_nonzero(y-y_test)/N

	def predict_proba(self, X_test):
		x=Variable(X_test)
		y=self.model.predictor(x)
		y=np.exp(y.data)
		H=y.sum(1).reshape(-1,1)
		return np.exp(y)/H


In [22]:
from google.colab import drive
drive.mount('/content/gdrive')


# モデル設定
batch_size = 100  # バッチサイズ
n_epoch = 100  # エポック数
n_channel = 3  # channel数（画像の奥行的な。カラー画像ならRGBなので3、モノクロなら1）
n_label = 4  # 正解ラベルの種類数


# 定義
clf = DeepLearningClassifier()
# トレーニング
clf.fit(X_train, Y_train)
# 予測
prediction = clf.predict(X_test)
# 精度測定
acc = clf.score(X_test, Y_test)

"""
トレーニングと予測精度の測定を一度にやってしまいたい場合は
clf.fit_and_score(X_train, y_train, X_test, y_test)
各ラベルの確率を計算したいなら
clf.predict_proba(X_test)
"""

serializers.save_npz("gunmanChainer.npz", model)
!cp gunmanChainer.npz /content/gdrive/My\ Drive/



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
epoch       main/accuracy
1           0.273333       
2           0.32875        
3           0.3425         
4           0.47125        
5           0.4325         
6           0.455          
7           0.48625        
8           0.48           
9           0.48375        
10          0.46375        
11          0.4975         
12          0.55125        
     total [######............................................] 12.39%
this epoch [###################...............................] 39.16%
       100 iter, 12 epoch / 100 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
13          0.55875        
14          0.5525         
15          0.595556       
16          0.58           
17          0.60875        
18          0.655          
19          0.685          
20          0.63375        
21          0.6025         
22          0.662

In [19]:
!ls -l


total 15764
-rw-r--r-- 1 root root  1544811 Mar 11 23:18 gunmanChainer.npz
drwxr-xr-x 6 root root     4096 Mar 11 22:49 images
-rw-r--r-- 1 root root 14575997 Jan 14 12:40 images.zip
drwxr-xr-x 2 root root     4096 Mar 11 23:16 result
drwxr-xr-x 1 root root     4096 Mar  8 17:26 sample_data


ーーーーーーーーーーーーーーー ここから下は、ごみ ーーーーーーーーーーーーーーーーーーーー

In [0]:
correct = 0
for i in range(len(Y_train)):
    x = Variable(np.array([Y_train[i][0]],dtype=np.float32))
    t = Y_train[i][1]
    y = model.predict(x)

    index = np.argmax(y.data)
    if index == t:
        correct += 1

print("正解：", correct ,"　全体：", len(test_data), "　正解率：", correct / len(test_data) * 100,"%")

In [0]:
class Gazou(Chain):
    def __init__(self):
        super(Gazou,self).__init__(
            cnn1 = L.Convolution2D(3,15,5),    #(15,50-(5-1)x50-(5-1))=(15,46x46)
            cnn2 = L.Convolution2D(15,40,5),   #(40,19x19)
            l1 = L.Linear(4000,100),          #40x10x10=4000
            l2 = L.Linear(100,4),
        )

    def ____call__(self, x, t):
        return F.softmax_cross_entropy(self.predict(x), t)

#    def predict(self,x):
    def __call__(self,x):
        h1 = F.max_pooling_2d(F.relu(self.cnn1(x)), 2)  #(15,23x23)
        h2 = F.max_pooling_2d(F.relu(self.cnn2(h1)), 2) #(40,10x10)
        h3 = F.dropout(F.relu(self.l1(h2)))
        return self.l2(h3)

model = Gazou()
opt = optimizers.Adam()
opt.setup(model)

iterator = iterators.SerialIterator(X_train,10)
updater = training.StandardUpdater(iterator, opt)
trainer = training.Trainer(updater,(20, "epoch"))
trainer.extend(extensions.ProgressBar())
trainer.run()

serializers.save_npz("gunmanChainer.npz", model)

In [0]:
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_mldata
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import optimizers, Chain, Variable, initializers
from chainer.training import extensions
from chainer.datasets import tuple_dataset

# モデル設定
batch_size = 100  # バッチサイズ
n_epoch = 20  # エポック数
n_channel = 1  # channel数（画像の奥行的な。カラー画像ならRGBなので3、モノクロなら1）
n_label = 10  # 正解ラベルの種類数

class MLP(Chain):
	# 多層パーセプトロンによる分類
	def __init__(self):
		super(MLP, self).__init__()
		with self.init_scope():
			self.fc1 = L.Linear(None, 100)
			self.fc2 = L.Linear(100, n_label)
			self.bn1 = L.BatchNormalization(100)

	def __call__(self, x):
		h = F.sigmoid(self.fc1(x))
		h = self.bn1(h)
		return self.fc2(h)

class LeNet(Chain):
	# CNNを用いた分類
	def __init__(self):
		super(LeNet, self).__init__()
		with self.init_scope():
			self.conv1 = L.Convolution2D(n_channel,6,5,1)
			self.conv2 = L.Convolution2D(6,16,5,1)
			self.conv3 = L.Convolution2D(16,120,4,1)
			self.fc4 = L.Linear(None, 84)
			self.fc5 = L.Linear(84,n_label)

	def __call__(self, x):
		h = F.sigmoid(self.conv1(x))
		h = F.max_pooling_2d(h, 2, 2)
		h = F.sigmoid(self.conv2(h))
		h = F.max_pooling_2d(h,2,2)
		h = F.sigmoid(self.conv3(h))
		h = F.sigmoid(self.fc4(h))
		return self.fc5(h)

class Alex(Chain):
	# AlexNet
	def __init__(self):
	    super(Alex, self).__init__(
	        conv1 = L.Convolution2D(n_channel, 96, 11, stride=4),
	        conv2 = L.Convolution2D(96, 256, 5, pad=2),
	        conv3 = L.Convolution2D(256, 384, 3, pad=1),
	        conv4 = L.Convolution2D(384, 384, 3, pad=1),
	        conv5 = L.Convolution2D(384, 256, 3, pad=1),
	        fc6 = L.Linear(None, 4096),
	        fc7 = L.Linear(4096, 4096),
	        fc8 = L.Linear(4096, n_label),
	    )

	def __call__(self, x):
		h = F.max_pooling_2d(F.local_response_normalization(
		    F.relu(self.conv1(x))), 3, stride=2)
		h = F.max_pooling_2d(F.local_response_normalization(
		    F.relu(self.conv2(h))), 3, stride=2)
		h = F.relu(self.conv3(h))
		h = F.relu(self.conv4(h))
		h = F.max_pooling_2d(F.relu(self.conv5(h)), 2, stride=2)
		h = F.dropout(F.relu(self.fc6(h)))
		h = F.dropout(F.relu(self.fc7(h)))
		return self.fc8(h)

class DeepLearningClassifier:
	def __init__(self):
		model = Alex()
		self.model = L.Classifier(model)
		self.opt = optimizers.Adam()
		self.opt.setup(self.model)

	def fit(self,X_train, y_train):
		train_data = tuple_dataset.TupleDataset(X_train, y_train)
		train_iter = chainer.iterators.SerialIterator(train_data, batch_size)
		updater = chainer.training.StandardUpdater(train_iter, self.opt)
		self.trainer = chainer.training.Trainer(updater, (n_epoch, 'epoch'), out='result')
		self.trainer.extend(extensions.LogReport())
		self.trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy']))
		self.trainer.extend(extensions.ProgressBar())
		self.trainer.run()

	def fit_and_score(self, X_train, y_train, X_test, y_test):
		train_data = tuple_dataset.TupleDataset(X_train, y_train)
		test_data = tuple_dataset.TupleDataset(X_test, y_test)
		train_iter = chainer.iterators.SerialIterator(train_data, batch_size)
		test_iter = chainer.iterators.SerialIterator(test_data, batch_size, repeat=False, shuffle=False)
		updater=chainer.training.StandardUpdater(train_iter, self.opt)
		self.trainer = chainer.training.Trainer(updater, (n_epoch, 'epoch'), out='result')
		self.trainer.extend(extensions.Evaluator(test_iter, self.model))
		self.trainer.extend(extensions.LogReport())
		self.trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy']))
		self.trainer.extend(extensions.ProgressBar())
		self.trainer.run()

	def predict(self, X_test):
		x=Variable(X_test)
		y=self.model.predictor(x)
		answer=y.data
		answer=np.argmax(answer, axis=1)
		return answer

	def score(self, X_test, y_test):
		y=self.predict(X_test)
		N=y_test.size
		return 1.0-np.count_nonzero(y-y_test)/N

	def predict_proba(self, X_test):
		x=Variable(X_test)
		y=self.model.predictor(x)
		y=np.exp(y.data)
		H=y.sum(1).reshape(-1,1)
		return np.exp(y)/H

if __name__=='__main__':
	# mnist 使用例
	# 前処理
	mnist = fetch_mldata('MNIST original', data_home=".")
	X = mnist.data
	y = mnist.target
	X =X/X.max()
	X = X.astype(np.float32)
	y = y.astype(np.int32)
	X =X .reshape(70000,1,28,28)  # 必ず（データの総数, channel数, 縦, 横）の形にしておく
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

	# 定義
	clf = DeepLearningClassifier()
	# トレーニング
	clf.fit(X_train, y_train)
	# 予測
	prediction = clf.predict(X_test)
	# 精度測定
	acc = clf.score(X_test, y_test)

	"""
	トレーニングと予測精度の測定を一度にやってしまいたい場合は
	clf.fit_and_score(X_train, y_train, X_test, y_test)
	各ラベルの確率を計算したいなら
	clf.predict_proba(X_test)
	"""